In [1]:
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
import alpaca_trade_api as tradeapi
from finta import TA
import pickle

# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [2]:
with open('gbt_classifier.pkl', 'rb') as f:
    papertrading_model = pickle.load(f)
    
#papertrading_model = xgb.XGBClassifier()
#papertrading_model.load_model('xgb_clf.bst')
scaler = pickle.load(open('scaler_model.pkl','rb'))



In [3]:
feats = ['HMA_5', 'RSI_5', 'ATR_14', 'RSI_14', 'RSI_150', 'cci']

In [4]:
# Load environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

In [5]:
print(f'The Alpaca public key used by the REST API is : {alpaca_api_key}')

The Alpaca public key used by the REST API is : PK2RJWWRGDVPCFAT43MM


In [6]:
alpaca_api = tradeapi.REST(alpaca_api_key,alpaca_secret_key, 'https://paper-api.alpaca.markets')

In [7]:
data_trade = alpaca_api.get_crypto_bars(['BTC/USDT'], tradeapi.TimeFrame.Hour, "2023-01-01").df

In [8]:
for time in [5,14,150]:
    data_trade['RSI_'+str(time)] = TA.RSI(data_trade, time)
data_trade['ATR_14'] = TA.ATR(data_trade, 14)
data_trade['HMA_5'] = TA.HMA(data_trade, 5)
data_trade['cci'] = TA.CCI(data_trade)

In [9]:
data_trade.dtypes

close          float64
high           float64
low            float64
trade_count      int64
open           float64
volume         float64
vwap           float64
symbol          object
RSI_5          float64
RSI_14         float64
RSI_150        float64
ATR_14         float64
HMA_5          float64
cci            float64
dtype: object

In [10]:
# transform preprocessed data to numpy array
data_to_predict = scaler.transform(data_trade[feats].dropna())

In [11]:
data_to_predict[-1]

array([ 1.06439536e+02, -1.99557112e-01,  3.29848206e+01,  1.04524215e-02,
        1.71257355e+00, -4.63454904e-01])

In [12]:
# make predictions using the model
predictions = papertrading_model.predict(data_to_predict)

In [13]:
BUY = predictions[-1]
display(BUY)

0.0

In [29]:
# Get the current price of BTC in USD
eth_usd_price = alpaca_api.get_latest_crypto_orderbook(['ETH/USD'])['ETH/USD'].asks[0].p

# Calculate the quantity of BTC to buy
usd_balance = float(alpaca_api.get_account().cash)
quantity_to_buy = usd_balance / eth_usd_price

In [32]:
# Get the current price of BTC in USD
eth_usd_price = alpaca_api.get_latest_crypto_orderbook(['ETH/USD'])['ETH/USD'].asks[0].p

    # Calculate the quantity of BTC to buy or sell
usd_balance = float(alpaca_api.get_account().cash)
quantity_to_trade = usd_balance / eth_usd_price * 0.1

    # Define order parameters
symbol = 'ETHUSD'
order_type = 'limit'
limit_price = eth_usd_price  # limit price for the order
time_in_force = 'gtc'
qty = quantity_to_trade  # quantity of ETH to trade

    # check if BUY value changed from previous time
if 'previous_buy' not in locals():
    previous_buy = BUY

if BUY != previous_buy:
    if BUY:
        side = 'buy'
        limit_price += 0.0001
    else:
        side = 'sell'
        qty = float(alpaca_api.get_position(symbol).qty)  # quantity of ETH to sell (same as the current position)
        limit_price -= 0.0001

        # Place order
    try:
        order = alpaca_api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type=order_type,
            time_in_force=time_in_force,
            limit_price=limit_price
        )
        print(f"{side.capitalize()} order for {qty} {symbol} at {limit_price} submitted successfully.")

    except Exception as e:
        print(f"Error submitting {side.capitalize()} order: ", e)

        # Update previous_buy value
    previous_buy = BUY
else :
    print('.', end='')

.

In [33]:
eth_usd_price

1759.122

In [34]:
quantity_to_buy

53.26621345753518

In [35]:
def trade_in_alpaca(quantity, tries=0):
    if papertrading_model.predict(data_to_predict)[-1]==1 and papertrading_model.predict(data_to_predict)[-2]==0:
        try:
            alpaca_api.submit_order(symbol='BTC/USDT',qty=0.04,side="buy",type='market', time_in_force='ioc')
        except Exception as e:
            if 'insufficient balance' in str(e) and tries<=10:
                print(quantity, tries)
                trade_in_alpaca(quantity=0.9*quantity, tries=tries+1)
            else:
                print(e)

In [19]:
trade_in_alpaca(quantity=quantity_to_buy)

In [20]:
alpaca_api.list_positions()

[Position({   'asset_class': 'crypto',
     'asset_id': '35f33a69-f5d6-4dc9-b158-4485e5e92e4b',
     'asset_marginable': False,
     'avg_entry_price': '1726.9389622201723257',
     'change_today': '-0.0387440192044649',
     'cost_basis': '34538.259729355441819006',
     'current_price': '1749.88',
     'exchange': 'FTXU',
     'lastday_price': '1820.41',
     'market_value': '34997.0735835996',
     'qty': '19.99969917',
     'qty_available': '4.183035257',
     'side': 'long',
     'symbol': 'ETHUSD',
     'unrealized_intraday_pl': '-104.3196048014504115240883194',
     'unrealized_intraday_plpc': '-0.0029719505502682',
     'unrealized_pl': '458.813854244158180994',
     'unrealized_plpc': '0.0132842203932526'})]

In [21]:
import xgboost as xgb
import alpaca_trade_api as tradeapi
from finta import TA
import pickle
import os
import time
from dotenv import load_dotenv
import pandas as pd

# Load environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Import saved XGB model
with open('gbt_classifier.pkl', 'rb') as f:
    papertrading_model = pickle.load(f)
    
scaler = pickle.load(open('scaler_model.pkl','rb'))

# Load scaler model
scaler = pickle.load(open('scaler_model.pkl', 'rb'))

# Define list of features
feats = ['HMA_5', 'RSI_5', 'ATR_14', 'RSI_14', 'RSI_150', 'cci']

# Initialize Alpaca API
alpaca_api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, 'https://paper-api.alpaca.markets')


# Get latest data on 1HR timeframe
data_trade = alpaca_api.get_crypto_bars(['BTC/USDT'], tradeapi.TimeFrame.Hour, "2023-01-01").df

# Compute the features needed for the prediction
for time in [5, 14, 150]:
    data_trade['RSI_'+str(time)] = TA.RSI(data_trade, time)
data_trade['ATR_14'] = TA.ATR(data_trade, 14)
data_trade['HMA_5'] = TA.HMA(data_trade, 5)
data_trade['cci'] = TA.CCI(data_trade)

# Transform preprocessed data to numpy array
data_to_predict = scaler.transform(data_trade[feats].dropna())

# Make predictions using the model
predictions = papertrading_model.predict(data_to_predict)

# Check the trade direction: BUY (=1) or SELL (=0)
BUY = predictions[-1]

# Get the current price of BTC in USD
eth_usd_price = alpaca_api.get_latest_crypto_orderbook(['ETH/USD'])['ETH/USD'].asks[0].p

# Calculate the quantity of BTC to buy or sell
usd_balance = float(alpaca_api.get_account().cash)
quantity_to_trade = usd_balance / eth_usd_price * 0.1

# Define order parameters
symbol = 'ETHUSD'
order_type = 'limit'
limit_price = eth_usd_price  # limit price for the order
time_in_force = 'gtc'
qty = quantity_to_trade  # quantity of ETH to trade

if BUY:
    side = 'buy'
    limit_price += 0.0001
else:
    side = 'sell'
    qty = float(alpaca_api.get_position(symbol).qty)  # quantity of ETH to sell (same as the current position)
    limit_price -= 0.0001

# Place order
try:
    order = alpaca_api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type=order_type,
        time_in_force=time_in_force,
        limit_price=limit_price
    )
    print(f"{side.capitalize()} order for {qty} {symbol} at {limit_price} submitted successfully.")

except Exception as e:
    print(f"Error submitting {side.capitalize()} order: ", e)



Error submitting Sell order:  insufficient balance for ETH (requested: 19.99969917, available: 4.183035257)


In [22]:
print('A', end='')
print('A', end='')
print('A', end='')
print('A', end='')
print('A', end='')

AAAAA

In [23]:
# Check if the API is running
try:
    account_info = alpaca_api.get_account()
    print(f"API is running! Status is {account_info.status} for account #{account_info.account_number} ")
except Exception as e:
    print("Error getting account info:", e)

API is running! Status is ACTIVE for account #PA33SNGXN8PD 


In [24]:
account_info



Account({   'account_blocked': False,
    'account_number': 'PA33SNGXN8PD',
    'accrued_fees': '0',
    'balance_asof': '2023-03-17',
    'bod_dtbp': '0',
    'buying_power': '132052.073155019068234666',
    'cash': '66026.036577509534117333',
    'created_at': '2022-06-05T15:16:17.049411Z',
    'crypto_status': 'ACTIVE',
    'crypto_tier': 2,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'effective_buying_power': '132052.073155019068234666',
    'equity': '101023.110161109134117333',
    'id': '1db5bc60-d786-4889-8085-9b2adc0fe4b2',
    'initial_margin': '0',
    'last_equity': '101800.21333089512',
    'last_maintenance_margin': '0',
    'long_market_value': '34997.0735835996',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '66026.03',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '101023.110161109134117333',
    'position_market_value': '34997.0735835996',
 

In [25]:
print(account_info.status)

ACTIVE
